<a id="top"></a>
### Content Miner Data
***

This notebook combines crawled data from Watson Discovery with data provided by MSK specifically to be used in Watson Discovery Content Miner. 

The MSK Analytics Data includes URL path information for the top 100 most visited web pages on the MSK Benefits site along with key metrics such as "Average time on page", "Unique views", etc. The Watson Discovery crawled data contains URL, text, PDF, content from tables (and more).

The purpose of combining these two datasets is to provide a rich source of data that can be examined and anlyzed in-depth using the Content Miner tool.

The notebook is organized as follows:

1. Load data and conduct EDA
2. Create "key" column for merging data
3. Merge datasets
4. Create new feature
5. Clean up final dataframe for Content Miner
6. Content Miner Findings

<div class="alert-warning">   
   
<font size="3"><b>*Please note*</b>: The first code block below assumes you have a CSV file of Watson Discovery scraped data available under the **Data Assets** section of your project. 
    
If you do not have scraped data available to import as a CSV, please go to the notebook titled **1_WatsonDiscovery_Crawl_and_Clean.ipynb** and run the respective code.</font>  
    

</div>

### Load Data and EDA

We will start by importing both sets of data which are currently under the Data Assets section for the project. In addition, we will load several Python libraries used for clean-up, feature creation, and merging dataframes. We will then conduct some preliminary exploratory data analysis (EDA) to look at the first two rows of data, the shape of the dataframe, how many values are null by column, etc.

### Watson Discovery Scraped Data
***

In [1]:
from project_lib import Project
project = Project.access()

# Fetch the file
watson_discovery_500 = project.get_file("msk_benefits_500_crawled_CLEAN_data.csv")
watson_discovery_500.seek(0)

msk_pages = project.get_file('MSK_pages_data.csv')
msk_pages.seek(0)


import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 1000)

df = pd.read_csv(watson_discovery_500)
df1 = pd.read_csv(msk_pages)

### Review columns, shape, isnull values

In [2]:
df.columns

Index(['document_id', 'result_metadata', 'enriched_text', 'metadata',
       'extracted_metadata', 'html', 'text', 'title', 'document_passages',
       'width', 'provider_url', 'type', 'provider_name', 'version', 'height',
       'date', 'parent', 'menu_order', 'modified_gmt', 'author', 'link',
       'comment_status', 'content', 'featured_media', 'ping_status', 'guid',
       'modified', 'excerpt', 'date_gmt', 'slug', 'status', 'name', 'count',
       'taxonomy', 'description', 'template'],
      dtype='object')

In [3]:
df.shape

(2921, 36)

In [4]:
df.isnull().sum()

document_id             38
result_metadata       2301
enriched_text         2489
metadata              2460
extracted_metadata    2505
html                  2584
text                  2604
title                 2601
document_passages     2550
width                 2895
provider_url          2897
type                  2888
provider_name         2897
version               2897
height                2897
date                  2908
parent                2904
menu_order            2908
modified_gmt          2908
author                2908
link                  2904
comment_status        2908
content               2908
featured_media        2908
ping_status           2908
guid                  2908
modified              2910
excerpt               2910
date_gmt              2910
slug                  2906
status                2910
name                  2915
count                 2915
taxonomy              2915
description           2918
template              2918
dtype: int64

### Drop null values in text column

Below we are going to drop null values in the text column. Because one of the main fields to be analyzed in Content Miner is the text from the web pages, it makes sense to drop any rows where there is no data in the text field.

In [5]:
df = df.dropna(subset=['text'])

### Fill empty "title" cells

In the "isnull().sum()" command above, we see that there are 26 empty title fields. We don't want to drop these because it may cause us to lose some populated text fields. We will fill these empty title cells with a string "NotAvailable" to remove null values but not lose important data.

In [6]:
df['title'] = df['title'].fillna('NotAvailable')

### MSK Analytics Data
***

Now let's look at the MSK Analytics Data. Note this data is in "df1". We will conduct high-level EDA by looking at the first two rows of data, reviewing the data shape, and looking at the total number of null values by column.

In [7]:
df1.head(2)

Page_Content                                     Page  \
0                   landing page                                        /   
1  open enrollment overview 2021  /keyword/open-enrollment-overview-2021/   

                    Group Content-Type                 Page_Content.1    Type  \
0  medical&life-insurance    home page                   landing page  static   
1               new-hires   enrollment  open enrollment overview 2021  search   

       Tile  pageviews  unique_pageviews  avg_time_on_page  entrances  \
0      Home     116711             87164             51.60      81885   
1  New Hire      27192             16891             92.51       4871   

   bounce_rate  percent_exit  page_value  
0         0.25          0.24           0  
1         0.23          0.16           0

In [8]:
df1.shape

(100, 14)

In [9]:
df1.isnull().sum()

Page_Content        0
Page                0
Group               0
Content-Type        0
Page_Content.1      0
Type                0
Tile                1
pageviews           0
unique_pageviews    0
avg_time_on_page    0
entrances           0
bounce_rate         0
percent_exit        0
page_value          0
dtype: int64

### Fill empty Tile cells

Similar to the empty "title" cells in the Watson Discovery scraped data, there are few empty "Tile" cells. The "Tile" column was manually created by IBM for early EDA. IBM assigned a Tile for each URL path in the MSK Analytics Data. Because we don't want to lose any additional rows of data, we will fill the null value in the Tile column with a string "NotAvailable".

In [10]:
df1['Tile'] = df1['Tile'].fillna('NotAvailable')

In [11]:
df1.isnull().sum()

Page_Content        0
Page                0
Group               0
Content-Type        0
Page_Content.1      0
Type                0
Tile                0
pageviews           0
unique_pageviews    0
avg_time_on_page    0
entrances           0
bounce_rate         0
percent_exit        0
page_value          0
dtype: int64

### Create "Key" Column for Merging Data

Because neither dataset has a common column of data on which to merge, we need to create one. The MSK Analytics Data has a column of URL paths and the Watson Discovery data has a column of full URLs. 

A list of strings will be created from the MSK Analytics URL path data. Then, the Watson Discovery "cleaned_url" column will be compared with the list of path strings. If a string is located in the cleaned_url, the path will be appended as a new column in the Watson Discovery data.

By doing this, both dataframes will have a column called "Page". This will be the column on which the two datasets are merged.

#### Review data in Page column

In [12]:
df1.Page.head(5)

0                                          /
1    /keyword/open-enrollment-overview-2021/
2       /keyword/2020-medical-plan-overview/
3                                 /loa-tool/
4              /keyword/sick-and-disability/
Name: Page, dtype: object

### Create list of strings based on URL paths from MSK data
The 'print' statement is commented out to avoid a lengthy notebook. Feel free to un-comment it and run the notebook to review the list.

In [13]:
match_list = df1['Page'].astype(str).values.tolist()
#print(match_list)

### Clean URLs in Watson Discovery scraped json data

The URLs in the Watson Discovery data must be cleaned. URL's are inside the metadata as a string\

In [14]:
df.metadata.head(2)

0    {'parent_document_id': '165aec4a7d54601ae8a5fe6949046eae58813c7bb8547ce997b080eb894c91b7', 'source': {'url': 'https://mskbenefits.mskcc.org/keyword/looking-for-child-care/'}, 'Date': 1626956951842, 'FileSize': 58872}
1       {'parent_document_id': '8a965f3957f00de773318927c62a1528c6b0250b8090a4e83d35f4c3831edb7e', 'source': {'url': 'https://mskbenefits.mskcc.org/keyword/adoption-assistance/'}, 'Date': 1626957112301, 'FileSize': 39418}
Name: metadata, dtype: object

### Function to clean URLs

In [15]:
def urlclean(x):
    start_index = x.find("url': ")+7
    end_index = x.find("Date")-5
    return x[start_index:end_index]

### Apply function to URL

In [16]:
df['cleaned_url'] = df['metadata'].astype(str).apply(lambda x: urlclean(x))

### Review output; ensure data type is string

In [17]:
df.cleaned_url = df.cleaned_url.astype(str)

In [18]:
df.cleaned_url.head(2)

0    https://mskbenefits.mskcc.org/keyword/looking-for-child-care/
1       https://mskbenefits.mskcc.org/keyword/adoption-assistance/
Name: cleaned_url, dtype: object

### Run 'cleaned_url' through loop that creates new column called 'Page'.

- Run 'cleaned_url' column through loop that looks for any of the URL paths (from above list) in the URL
- Create new column called 'Page' which will resemble the contents of the MSK Analytics data column called 'Page'
- These two columns will be used to conduct the data merge

In [19]:
df['Page'] = df['cleaned_url'].apply(lambda x: ''.join([i for i in match_list if i in x]))

### Review new column in df

We review the new column to make sure it matches the syntax of the 'Page' column in the MSK data. There is an extra '/' at the beginning of each row that needs to be removed.

In [20]:
df.Page.head(10)

0       //keyword/looking-for-child-care/
1                                       /
2                                       /
3                                       /
4                                       /
5                                       /
6        //keyword/leave-of-absence-faqs/
7                                       /
8     //keyword/fitness-center-discounts/
26                                      /
Name: Page, dtype: object

### Remove the extra '/' at the beginning  and review output

In [21]:
df.Page.astype(str)
df['Page'] = df['Page'].replace({'^\/':''}, regex=True)
print(df['Page'].head(10))

0       /keyword/looking-for-child-care/
1                                       
2                                       
3                                       
4                                       
5                                       
6        /keyword/leave-of-absence-faqs/
7                                       
8     /keyword/fitness-center-discounts/
26                                      
Name: Page, dtype: object


### Move 'Page' column in both dataframes
Move the 'Page' column in each dataframe to be the first column in the dataframe. This is necessary when implementing 'merge' because the second dataframe (MSK Analytics) will append the column to the existing Watson Discovery dataframe.

In [22]:
first_column = df.pop('Page')
first_column1 = df1.pop('Page')

df.insert(0, 'Page', first_column)
df1.insert(0, 'Page', first_column1)

### Confirm first col is 'Page' for MSK data

In [23]:
df1.head(1)

Page  Page_Content                   Group Content-Type Page_Content.1  \
0    /  landing page  medical&life-insurance    home page   landing page   

     Type  Tile  pageviews  unique_pageviews  avg_time_on_page  entrances  \
0  static  Home     116711             87164              51.6      81885   

   bounce_rate  percent_exit  page_value  
0         0.25          0.24           0

### Merge two dataframes on 'Page' and call resulting dataframe 'final'

In [24]:
final = pd.merge(df, df1, on='Page')

### EDA and Clean-up

We will do some high-level EDA and clean-up on the 'final' dataframe since this file will be output as a csv to upload in Content Miner. We'll look at the shape, first two rows, and columns to drop unnecessary fields.

In [25]:
final.shape

(85, 51)

In [26]:
final.columns

Index(['Page', 'document_id', 'result_metadata', 'enriched_text', 'metadata',
       'extracted_metadata', 'html', 'text', 'title', 'document_passages',
       'width', 'provider_url', 'type', 'provider_name', 'version', 'height',
       'date', 'parent', 'menu_order', 'modified_gmt', 'author', 'link',
       'comment_status', 'content', 'featured_media', 'ping_status', 'guid',
       'modified', 'excerpt', 'date_gmt', 'slug', 'status', 'name', 'count',
       'taxonomy', 'description', 'template', 'cleaned_url', 'Page_Content',
       'Group', 'Content-Type', 'Page_Content.1', 'Type', 'Tile', 'pageviews',
       'unique_pageviews', 'avg_time_on_page', 'entrances', 'bounce_rate',
       'percent_exit', 'page_value'],
      dtype='object')

### Keep relevant columns
For simplicity, we will only keep relevant columns for the output file. However, Content Miner offers the flexibility to choose which columns to show. So if a large file is uploaded to Content Miner, a user can select only those columns needed.

In [27]:
final = final[['Page','text', 'title', 'cleaned_url','Page_Content','Group','Content-Type','Type', 'Tile','pageviews',
              'unique_pageviews','avg_time_on_page', 'entrances', 'bounce_rate', 'percent_exit']]

### Create New Feature

Because the keywords in the URL path could be useful for an analysis, we will create a new column called "url_keywords". If a url path contains the word "/keyword/", words following "keyword" will be appended as a new column.  

### Regex pattern to find '/keyword/' in URLs

In [28]:
pattern_ = re.compile(r'^\/keyword\/(.*)\/')

### Extract keywords in URLs

The below loops finds the pattern '/keyword/' from the 'Page' column (URL path from MSK), extracts all the words after it up until the next '/', then removes the dashes.

In [29]:
final['Page'] = final['Page'].astype(str)

_key_word = []

for page in final['Page']:
    if '/keyword/' in page:
        #print(z)
        temp_ = pattern_.search(page)
        temp2_ = temp_.group(1).replace("-", ' ')
        _key_word.append(temp2_)
    else:
        _key_word.append('NA')


### Add column of keywords to dataframe

In [30]:
final['url_keywords'] = _key_word

### Review keyword column for accuracy

In [31]:
final[['Page','url_keywords']].head(5)

Page  \
0                                                                                                                /keyword/looking-for-child-care/   
1                                                                                                                 /keyword/leave-of-absence-faqs/   
2                                                                                                              /keyword/fitness-center-discounts/   
3  /keyword/check-your-providers-metlife/?childslug=check-your-providers-metlife&syn=MetLife+Providers&parentslug=dental-parent&parentname=Dental   
4                                                                                                                               /keyword/be-well/   

                   url_keywords  
0        looking for child care  
1         leave of absence faqs  
2      fitness center discounts  
3  check your providers metlife  
4                       be well

### Clean Text Column

Now we need to clean the text column from white space, irrelevant alpha patterns, and foreign characters.

- Ensure 'text' column is string format
- Print full text column for manual review and look for anomalies; this is a very manual process but necessary for good NLP
- The Print statement is commented out due to how long it makes the notebook); feel free to un-comment it for review

In [32]:
final['text'] = final['text'].astype(str)
#print(final['text'])

### Clean 'text' column

The following reduces 2 white spaces to one, removes foreign characters and "noisy" alpha patterns, and removes the brackets and quotation marks
that were included in the Watson Discovery scraped data. Creating a separate line for each change may appear inefficient, but after reviewing the cleaned data in the following code block, additional corrections were required.

In [33]:
final['text1'] = final['text'].astype(str).map(lambda x: re.sub("  ", " ", x))
final['text2'] = final['text1'].map(lambda x: re.sub('‚Äô', '\'', x))
final['text3'] = final['text2'].map(lambda x: re.sub('‚Äî', '', x))
final['text4'] = final['text3'].map(lambda x: re.sub('‚Äú', '', x))
final['text5'] = final['text4'].map(lambda x: re.sub('‚Äù', '', x))
final['text6'] = final['text5'].map(lambda x: re.sub('&nbsp;', ' ', x))
final['text7'] = final['text6'].map(lambda x: re.sub('‚Äì', ' ', x))
final['text8'] = final['text7'].map(lambda x: re.sub('&gt;', '', x))
final['text9'] = final['text8'].map(lambda x: re.sub('\[\'', '', x))
final['text10'] = final['text9'].map(lambda x: re.sub("\'\]", '', x))
final['text11'] = final['text10'].map(lambda x: re.sub('\[\"', '', x))
final['text12'] = final['text11'].map(lambda x: re.sub("\"\]", '', x))

#### Un-comment to review cleaned text field

In [34]:
#print(final['text12'])

### Clean Title Column

Below we look at the "title" column and see if clean-up is required. Some foreign characters appear and those are removed.

In [35]:
final['title'][15]

'Dental ‚Äì MSK Benefits'

### Remove the foreign characters

In [36]:
final['title1'] = final['title'].astype(str).map(lambda x: re.sub(" ‚Äì", "", x))

### Review to make sure change was made

In [37]:
final['title1'][15]

'Dental MSK Benefits'

### Final clean-up

We will do one final review of the dataframe to make sure there are no null values in any column, identify and drop any columns that aren't needed after all the processing and text cleanup, and update the column names for two columns.

### Make sure 'final' dataframe has no null values in any columns

In [38]:
final.isnull().sum()

Page                0
text                0
title               0
cleaned_url         0
Page_Content        0
Group               0
Content-Type        0
Type                0
Tile                0
pageviews           0
unique_pageviews    0
avg_time_on_page    0
entrances           0
bounce_rate         0
percent_exit        0
url_keywords        0
text1               0
text2               0
text3               0
text4               0
text5               0
text6               0
text7               0
text8               0
text9               0
text10              0
text11              0
text12              0
title1              0
dtype: int64

### Drop columns created during cleanup

In [39]:
final = final.drop(['title','text','text1', 'text2', 'text3', 'text4','text5', 'text6', 'text7','text8', 'text9','text10','text11'], axis=1)

### Rename two columns
Rename 'text12' to 'text' column and rename 'title1' back to 'title; we are reverting back to the original column names after cleaning.

In [40]:
final = final.rename(columns={"text12": "text", 'title1':'title'})

### Review first few rows to ensure data is as expected

In [41]:
#Uncomment the below line to review, commented since output makes the notebook really big
#final.head(2)

### Save file to Project under 'Data Assets'

In [42]:
project.save_data("Final_ContentMiner.csv", final.to_csv(index=True), overwrite=True)

{'file_name': 'Final_ContentMiner.csv',
 'message': 'File saved to project storage.',
 'asset_id': 'aafc95dc-1dfe-4766-bf6d-6e4e9e7ae2b5'}

### Content Miner Findings
***

Several dashboards have been created in Content Miner based on the csv file above. Here are some highlights:

1. **URL Keywords, Content Miner Keywords**
- Employees visiting the HSA details web page spend some of the longest average time per page on the site.
- Among the Content Miner keywords, "money" ranked high for average time per page

2. **Average Time on Page Analysis**
- When the average time on page was combined with each of the Entities (Save, Retirement, Health), a 2.52 relevancy metric was identified for "contribution" for the Retirement entity.
- For the Save entity, a 2.69 relevancy metric occurred for "reimbursement" and "tax-free".

3. **3 or more Minutes (average time on page)**
- For employees who spent 3 or more minutes (metric Average Time on Page) on a given web page, the Content Type with the highest relevancy is "savings". 
- Page Details showed strong relevancy for "hsa details".



[Back to top](#top)